In [1]:
import pandas as pd
df=pd.read_csv('crypto_price_prediction_dataset.csv')

In [ ]:
df.head(5)

,O1,H1,L1,C1,V1,O2,H2,L2,C2,V2,...,L48,C48,V48,macd,macd_signal,rsi,sma20,sma50,sma200,target
0,46216.93,46731.39,46208.37,46656.13,1503.33095,46656.14,46949.99,46574.06,46778.14,943.81539,...,47255.38,47286.18,749.48136,-4.998207,9.570417,50.279883,47190.8480,NaN,NaN,9
1,46656.14,46949.99,46574.06,46778.14,943.81539,46778.14,46928.94,46721.96,46811.77,485.16860,...,47050.00,47050.08,734.11794,-16.372949,4.381744,47.182587,47197.8370,NaN,NaN,9
2,46778.14,46928.94,46721.96,46811.77,485.16860,46811.77,46916.63,46760.12,46813.20,562.88971,...,46950.00,47116.71,628.05580,-19.782984,-0.451202,47.608367,47199.6085,47175.0758,NaN,10
3,46811.77,46916.63,46760.12,46813.20,562.88971,46813.21,46887.33,46591.23,46711.05,861.88389,...,46858.44,46999.98,781.70714,-31.541012,-6.669164,44.239066,47193.4770,47181.9528,NaN,9
4,46813.21,46887.33,46591.23,46711.05,861.88389,46711.05,47555.55,46673.94,47192.55,1400.73642,...,46722.39,46793.26,850.43854,-56.884185,-16.712168,43.367195,47174.4675,47182.2552,NaN,8


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1. Загрузка данных
data = df

# 2. Подготовка данных
X = data.drop('target', axis=1).values  # Все кроме целевой переменной
y = data['target'].values  # Целевые классы

# Разделение на train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Нормализация данных (очень важно для GRU!)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Преобразование в 3D-формат для GRU [samples, timesteps, features]
# У нас 48 свечей × 5 параметров + 6 индикаторов = 246 фичей
# Но GRU ожидает последовательности, поэтому группируем по свечам:
X_train = X_train.reshape(X_train.shape[0], 48, 5)  # [примеры, 48 свечей, 5 параметров]
X_test = X_test.reshape(X_test.shape[0], 48, 5)

# 5. Создание модели GRU
model = Sequential([
    # Первый GRU-слой (возвращает последовательности для следующего слоя)
    GRU(128, input_shape=(48, 5), return_sequences=True),
    Dropout(0.2),  # Для борьбы с переобучением
    
    # Второй GRU-слой
    GRU(64, return_sequences=False),
    Dropout(0.2),
    
    # Выходной слой (13 нейронов = 13 классов)
    Dense(13, activation='softmax')
])

# 6. Компиляция модели
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 7. Callback для сохранения весов
checkpoint = ModelCheckpoint(
    'best_model_weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

# 8. Обучение модели
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=64,
    callbacks=[checkpoint]
)

# 9. Оценка модели
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nТочность на тестовых данных: {test_acc:.4f}")

ValueError: cannot reshape array of size 6072018 into shape (24683,48,5)